<details><summary style="display:list-item; font-size:16px; color:blue;">Jupyter Help</summary>
    
Having trouble testing your work? Double-check that you have followed the steps below to write, run, save, and test your code!
    
[Click here for a walkthrough GIF of the steps below](https://static-assets.codecademy.com/Courses/ds-python/jupyter-help.gif)

Run all initial cells to import libraries and datasets. Then follow these steps for each question:
    
1. Add your solution to the cell with `## YOUR SOLUTION HERE ## `.
2. Run the cell by selecting the `Run` button or the `Shift`+`Enter` keys.
3. Save your work by selecting the `Save` button, the `command`+`s` keys (Mac), or `control`+`s` keys (Windows).
4. Select the `Test Work` button at the bottom left to test your work.

![Screenshot of the buttons at the top of a Jupyter Notebook. The Run and Save buttons are highlighted](https://static-assets.codecademy.com/Paths/ds-python/jupyter-buttons.png)

**Setup**

We will start by reusing some code we went over in previous exercises: loading and splitting text data from files, storing the chunks in a Chroma collection with metadata, and setting up an OpenAI client. We then query the collection to retrieve the most relevant chunk for a given question.

    For our example documents, we'll be using the Wikipedia pages for three sci-fi/AI films: *Her*, *2001*, and *WALL-E*.
    
#### Note: The following cell is likely to take a minute to run.

In [1]:
import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI


chroma_client = chromadb.Client()

collection = chroma_client.get_or_create_collection(name="prompt_practice")

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", "? ", "! "],  # List of characters to split on
    chunk_size=2000,  # The maximum size of your chunks
    chunk_overlap=400,  # The maximum overlap between chunks
)

texts = [
    {"file": "2001.txt", "url": "https://en.wikipedia.org/wiki/2001:_A_Space_Odyssey", "title": "2001: A Space Odyssey"},
    {"file": "Her.txt", "url": "https://en.wikipedia.org/wiki/Her_(film)", "title": "Her"},
    {"file": "WALLE.txt", "url": "https://en.wikipedia.org/wiki/WALL-E", "title": "WALL-E"}
]

for idx, text in enumerate(texts):
    with open(text["file"], "r") as f:
        content = f.read()
    text['chunks'] = text_splitter.create_documents([content])
    for chunk_idx, chunk in enumerate(text['chunks']):
        collection.add(
            documents=[chunk.page_content],
            ids=[f"{idx}-{chunk_idx}"],
            metadatas={"chunk_idx": chunk_idx, "url": text["url"], "title": text['title']}

        )


client = OpenAI()

collection.query(query_texts=["Who did Kubrick partner with in 2001?"], n_results=1)

{'ids': [['0-24']],
 'distances': [[0.7828559279441833]],
 'metadatas': [[{'chunk_idx': 24,
    'title': '2001: A Space Odyssey',
    'url': 'https://en.wikipedia.org/wiki/2001:_A_Space_Odyssey'}]],
 'embeddings': None,
 'documents': [['The screenplay credits were shared whereas the *2001* novel, released shortly after the film, was attributed to Clarke alone. Clarke wrote later that "the nearest approximation to the complicated truth" is that the screenplay should be credited to "Kubrick and Clarke" and the novel to "Clarke and Kubrick".<sup id="cite_ref-Clarke_pp.31-38_40-0"><a href="https://en.wikipedia.org/wiki/2001:_A_Space_Odyssey#cite_note-Clarke_pp.31-38-40">[40]</a></sup> Early reports about tensions involved in the writing of the film script appeared to reach a point where Kubrick was allegedly so dissatisfied with the collaboration that he approached other writers who could replace Clarke, including [Michael Moorcock](https://en.wikipedia.org/wiki/Michael_Moorcock "Michael M

Now that we're set up, we can get started prompting.

Let's first create an easy way to prompt the LLM (in this case, GPT-4).

We define the function `get_completion`, which sends a user prompt and a system prompt to the OpenAI API to generate a completion using the specified model. The `system_prompt` is set to define the assistant's role as a helpful RAG search assistant.

In [2]:
def get_completion(user_prompt, system_prompt, model="gpt-4"):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return completion.choices[0].message.content


system_prompt = "You are a helpful RAG search assistant who uses results from a search engine to answer user queries."

RAG (Retrieval-Augmented Generation) uses a vector database to retrieve text chunks relevant to a query, which are then inserted into a prompt. The prompt is fed to a language model that generates an informed answer by incorporating the retrieved information. Let's see how to assemble the prompts now.

#### Checkpoint 1/3

Complete the `make_rag_prompt` function to generate a RAG prompt based on a given query and search results. It takes the following parameters.
- `query`: The user's question or query.
- `result_str`: The search results obtained from a document search, to be used as context for answering the query.

Inside the function, return a prompt using an f-string that includes the following components:
- Instructions for the model. We've written this one for you as an example, but read it carefully as a guide for writing your own, similar prompts in the future.
- The user's question.
- The search results.

Your task below is to pass the variables corresponding to the user's query and the stringified search results from the vector database underneath the lines `User question:` and `Search Results:` respectively.

Don't forget to run the cell and save the notebook before selecting `Test Work`! Open the `Jupyter Help` toggle at the top of the notebook for more details.

In [3]:
def make_rag_prompt(query, result_str):
  return f"""
Instructions:
Your task is to answer the following user question. The search results of a document search have been included to give you more context. Use the information in Search Results to help you answer the question accurately.
Not all information in the search results will be useful. However, if you find any information that's useful for answering the user's question, draw from it in your answer.

User question:
{query}


Search Results:
{result_str}


Your answer:
"""

#### Checkpoint 2/3

Now we'll take the RAG prompt our function outputs and build the rest of the RAG functionality around it in a new function.

Complete the `get_RAG_completion` function to retrieve relevant search results, format the RAG prompt, and generate a completion using the RAG prompt. It takes two parameters:

- `query`: The user's question or query.
- `n_results`: The number of search results to retrieve (default is 3).

Inside the function, perform the following steps:

- Query the Chroma collection to retrieve the relevant search results based on the user's query. Use the `collection.query` method with the appropriate parameters. Take a look at the example in the setup cell if you need to jog your memory. 

- Call the `make_rag_prompt` function, passing the user's query and the concatenated search results as arguments, to format the RAG prompt. It should be assigned to the `formatted_query` variable.

- Return the `get_completion` function, passing the formatted RAG prompt as the user prompt and use the existing `system_prompt` as the system prompt, to generate the final completion.


Don't forget to run the cell and save the notebook before selecting `Test Work`! Open the `Jupyter Help` toggle at the top of the notebook for more details.

In [4]:
def get_RAG_completion(query, n_results=3):
    ## YOUR SOLUTION HERE ##
    search_results = collection.query(query_texts = query , n_results = n_results)
    result_str = ""
    for result in search_results["documents"][0]:
        result_str += result
    ## YOUR SOLUTION HERE ##
    formatted_query = make_rag_prompt(query, result_str)
    print("\n********This is the RAG prompt********\n")
    print(formatted_query)
    print("\n*********************************\\n")
    return get_completion(formatted_query, system_prompt)

get_RAG_completion("What was the plot of Spike Jonze's 'Her'?")


********This is the RAG prompt********


Instructions:
Your task is to answer the following user question. The search results of a document search have been included to give you more context. Use the information in Search Results to help you answer the question accurately.
Not all information in the search results will be useful. However, if you find any information that's useful for answering the user's question, draw from it in your answer.

User question:
What was the plot of Spike Jonze's 'Her'?


Search Results:
# Her (film) - Wikipedia

> ## Excerpt
> 

---
[![This is a good article. Click here for more information.](https://upload.wikimedia.org/wikipedia/en/thumb/9/94/Symbol_support_vote.svg/19px-Symbol_support_vote.svg.png)](https://en.wikipedia.org/wiki/Wikipedia:Good_articles* "This is a good article. Click here for more information.")

From Wikipedia, the free encyclopedia

| Her |
| --- |
| [![](https://upload.wikimedia.org/wikipedia/en/4/44/Her2013Poster.jpg)](https://en.

"The plot of Spike Jonze's 'Her' revolves around Theodore Twombly, a lonely man in a near-future Los Angeles who works as a writer of personalized letters for people unable to express their emotions. The story unfolds as Theodore, who is going through the final stages of a painful divorce, becomes intrigued by a new, advanced operating system advertised as being uniquely intuitive and capable of evolving.\n\nHe decides to purchase the OS and during the setup he chooses a female identity for his virtual assistant. The AI names herself Samantha and is voiced by Scarlett Johansson. As Theodore and Samantha begin to interact, their relationship deepens, with Samantha's intelligence and sensitivity growing far beyond her initial programming.\n\nTheir unusual relationship explores themes of love, human connection, and the nature of consciousness. Over time, they fall in love, although Theodore struggles with the complexities of having feelings for an entity that does not have a physical pres

#### Checkpoint 3/3

Now you'll practice tweaking the prompt to coax the model into citing the sources it uses in its answer.

Append the following text at the end of the "Instructions" section in the prompt:
`At the end of your answer, cite the URL of the search result your answer draws from. Use the following format:
<Your answer here>. Source: <URL of the search result your answer comes from here>`

At the bottom of the cell, we call the same query we used in the last checkpoint. Compare it to the existing example to see how well the model managed to cite its sources.

Don't forget to run the cell and save the notebook before selecting `Test Work`! Open the `Jupyter Help` toggle at the top of the notebook for more details.

In [5]:
def make_cited_rag_prompt(query, result_str):
  return f"""
Instructions:
Your task is to answer the following user question. The search results of a document search have been included to give you more context. Use the information in Search Results to help you answer the question accurately.
Not all information in the search results will be useful. However, if you find any information that's useful for answering the user's question, draw from it in your answer.
At the end of your answer, cite the URL of the search result your answer draws from. Use the following format:
<Your answer here>. Source: <URL of the search result your answer comes from here>

User question:
{query}

Search Results:
{result_str}

Your answer:
"""

def get_cited_RAG_completion(query, n_results=3):
    ## YOUR SOLUTION HERE ##
    search_results = collection.query(query_texts=[query], n_results=n_results)
    result_str = ""
    for result in search_results["documents"][0]:
        result_str += result
    ## YOUR SOLUTION HERE ##
    formatted_query = make_cited_rag_prompt(query, result_str)
    print("\n********This is the cited RAG prompt********\n")
    print(formatted_query)
    print("\n*********************************\\n")
    return get_completion(formatted_query, system_prompt)

get_cited_RAG_completion("What was the plot of Spike Jonze's 'Her'?")


********This is the cited RAG prompt********


Instructions:
Your task is to answer the following user question. The search results of a document search have been included to give you more context. Use the information in Search Results to help you answer the question accurately.
Not all information in the search results will be useful. However, if you find any information that's useful for answering the user's question, draw from it in your answer.
At the end of your answer, cite the URL of the search result your answer draws from. Use the following format:
<Your answer here>. Source: <URL of the search result your answer comes from here>

User question:
What was the plot of Spike Jonze's 'Her'?

Search Results:
# Her (film) - Wikipedia

> ## Excerpt
> 

---
[![This is a good article. Click here for more information.](https://upload.wikimedia.org/wikipedia/en/thumb/9/94/Symbol_support_vote.svg/19px-Symbol_support_vote.svg.png)](https://en.wikipedia.org/wiki/Wikipedia:Good_articles* "T

"The plot of Spike Jonze's 'Her' revolves around Theodore Twombly (played by Joaquin Phoenix), a man who develops a relationship with Samantha (voiced by Scarlett Johansson), an artificially intelligent virtual assistant personified through a female voice. The story unfolds in a near-future setting and explores the complex nature of love and human relationships as Theodore's connection with Samantha becomes deeper and more emotionally significant. The film also features performances by Amy Adams, Rooney Mara, Olivia Wilde, and Chris Pratt. 'Her' is Jonze's solo screenwriting debut and is both a science-fiction and romantic drama film that provides insightful commentary on the state of modern human relationships and our interaction with technology. Source: https://en.wikipedia.org/wiki/Her_(film)"